# Add School Attendance Zones to Block Data
    

## Description of Program
- program:    IN-CORE_1hv1_AddSABS_BlockData
- task:       Add School Attendance Zones to Block Data
- Version:    2021-07-01
- project:    Interdependent Networked Community Resilience Modeling Environment (IN-CORE) Subtask 5.2 - Social Institutions
- funding:	  NIST Financial Assistance Award Numbers: 70NANB15H044 and 70NANB20H008 
- author:     Nathanael Rosenheim

- Suggested Citation:
Rosenheim, N. (2021) “Obtain, Clean, and Explore Labor Market Allocation Methods". 
Archived on Github and ICPSR.

In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import math as math
import numpy as np
import geopandas as gpd
import pandas as pd
import shapely
import descartes

import folium as fm # folium has more dynamic maps - but requires internet connection

import os # For saving output to path

In [2]:
# Display versions being used - important information for replication
import sys
print("Python Version     ", sys.version)
print("numpy version:     ", np.__version__)
print("geopandas version: ", gpd.__version__)
print("pandas version:    ", pd.__version__)
print("shapely version:   ", shapely.__version__)
# print("descartes version:   ", descartes.__version__)  1.1.0
print("folium version:    ", fm.__version__)

Python Version      3.7.10 | packaged by conda-forge | (default, Feb 19 2021, 15:37:01) [MSC v.1916 64 bit (AMD64)]
numpy version:      1.21.0
geopandas version:  0.9.0
pandas version:     1.2.5
shapely version:    1.7.1
folium version:     0.12.1


In [3]:
# Store Program Name for output files to have the same name
programname = "IN-CORE_1hv1_AddSABS_BlockData_2021-07-01"
# Make directory to save output
if not os.path.exists(programname):
    os.mkdir(programname)

### IN-CORE addons
This program uses coded that is being developed as potential add ons to pyincore. These functions are in a folder called pyincore_addons - this folder is located in the same directory as this notebook.
The add on functions are organized to mirror the folder sturcture of https://github.com/IN-CORE/pyincore

Each add on function attempts to follow the structure of existing pyincore functions and includes some help information.

In [4]:
# open, read, and execute python program with reusable commands
import pyincore_addons.geoutil_20210618 as add2incore

# since the geoutil is under construction it might need to be reloaded
from importlib import reload 
add2incore = reload(add2incore)

# Print list of add on functions
from inspect import getmembers, isfunction
print(getmembers(add2incore,isfunction))

[('df2gdf_WKTgeometry', <function df2gdf_WKTgeometry at 0x0000023E192EEB88>), ('nearest_pt_search', <function nearest_pt_search at 0x0000023E1D03B5E8>)]


## Read in Census Block Data
Census Blocks provide an estimate of how many residiential address points (housing units) should be located in each block.

In [5]:
help(add2incore.df2gdf_WKTgeometry)

Help on function df2gdf_WKTgeometry in module pyincore_addons.geoutil_20210618:

df2gdf_WKTgeometry(df: pandas.core.frame.DataFrame, projection='epsg:4326', reproject='epsg:4326', geometryvar='geometry')
    Function to convert dataframe with WKT Geometry to Geodata Frame
    
    Tested Python Enviroment:
        Python Version      3.7.10
        geopandas version:  0.9.0
        pandas version:     1.2.4
        shapely version:    1.7.1
    Args:
        :param df: dataframe with Well Known Text (WKT) geometry
        :param projection: String with Coordinate Reference System - default is epsg:4326
        :help projection: https://spatialreference.org/ref/epsg/wgs-84/
            Use UTM for measuring distances and area in meters
            Common Universal Transverse Mercator (UTM) for North America
            UTM zone 10N = West Coast     = epsg:26910
            UTM zone 17N = North Carolina = epsg:26917
            UTM zone 19N = Maine          = epsg:26919
            https

In [6]:
sourceprogram = "IN-CORE_1av2_Lumberton_CleanBlockData_2021-06-02"
filename = sourceprogram+"/"+sourceprogram+"EPSG4269.csv"
block_df = pd.read_csv(filename)

# Convert dataframe to gdf
block_gdf = add2incore.df2gdf_WKTgeometry(df = block_df, 
                                          projection = "epsg:4269",
                                          reproject="epsg:4326",
                                         geometryvar='rppnt4269')
block_gdf.head(2)

,Unnamed: 0,STATEFP10,COUNTYFP10,TRACTCE10,BLOCKCE,BLOCKID10,PARTFLG,HOUSING10,POP10,geometry,...,blockid,tothupoints,popcount,HU100,POP100,popdiff,PLCGEOID10,PLCNAME10,PUMGEOID10,PUMNAME10
0,0,37,155,961900,2028,371559619002028,N,14,52,POINT (-79.22459 34.45879),...,371559619002028,14,51,14,52,1,NaN,NaN,3705100,Robeson County (West)--Lumberton City PUMA
1,1,37,155,961900,2054,371559619002054,N,1,3,POINT (-79.18141 34.40608),...,371559619002054,1,3,1,3,0,NaN,NaN,3705100,Robeson County (West)--Lumberton City PUMA


In [7]:
block_gdf.columns

Index(['Unnamed: 0', 'STATEFP10', 'COUNTYFP10', 'TRACTCE10', 'BLOCKCE',
       'BLOCKID10', 'PARTFLG', 'HOUSING10', 'POP10', 'geometry',
       'CountySelect', 'rppnt4269', 'blk104269', 'blockid', 'tothupoints',
       'popcount', 'HU100', 'POP100', 'popdiff', 'PLCGEOID10', 'PLCNAME10',
       'PUMGEOID10', 'PUMNAME10'],
      dtype='object')

#### Note 
EPSG 4269 uses NAD 83 which will have slightly different lat lon points when compared to EPSG 4326 which uses WGS 84.

## Add School District ID To Blocks
### Read in School District polygons for state

The School Attendance Boundary Survey (SABS) was an experimental survey conducted by the National Center for Education Statistics (NCES) with assistance from the U.S. Census Bureau to collect school attendance boundaries for the 2013-2014 and 2015-2016 school years. The SABS collection includes boundaries for more than 70,000 schools in over 12,000 school districts throughout the U.S.

The SABS_1516 file is 556 MB. A previoud program selected the SABs for the County of Study.
The data has three overlapping layers - Hichschool, Middle School and Primary Schools. This section will add three new columns to the data one for each layer.

In [8]:
# Location of Unified School District Polygons Defined By US Census
sourcefolder = '../SourceData/nces.ed.gov/WorkNPR/'
sourceprogram = "NCES_2av1_SelectCountySchools_2021-06-06"
filename = sourcefolder+"/"+sourceprogram+"/"+"SABS_1516_37155_High.shp"
high_sabs_gdf = gpd.read_file(filename)
high_sabs_gdf.head()

,SrcName,ncessch,schnam,leaid,gslo,gshi,defacto,stAbbrev,openEnroll,Shape_Leng,Shape_Area,level,MultiBdy,slcncessch,slcleaid,geometry
0,Purnell Swett HS,370393002102,Purnell Swett High,3703930,09,12,0,NC,0,122991.586495,5.090427e+08,3,0,1,1,"POLYGON ((-79.21300 34.79015, -79.21441 34.779..."
1,S Robeson HS,370393002184,South Robeson High,3703930,09,12,0,NC,0,152761.549233,7.031522e+08,3,0,1,1,"POLYGON ((-79.28798 34.66740, -79.28630 34.666..."
2,Fairmont High,370393002232,Fairmont High,3703930,09,12,0,NC,0,150538.553654,7.662150e+08,3,0,1,1,"POLYGON ((-78.96800 34.55955, -78.96790 34.558..."
3,Lumberton HS,370393002237,Lumberton Senior High,3703930,09,12,0,NC,0,156694.540351,6.981158e+08,3,0,1,1,"POLYGON ((-78.84945 34.73266, -78.84932 34.732..."
4,Red Springs HS,370393002239,Red Springs High,3703930,09,12,0,NC,0,118402.139094,3.563852e+08,3,0,1,1,"POLYGON ((-79.07784 34.84129, -79.06428 34.833..."


In [9]:
high_sabs_gdf['ncessch'].describe()

count                6
unique               6
top       370393002244
freq                 1
Name: ncessch, dtype: object

In [10]:
filename = sourcefolder+"/"+sourceprogram+"/"+"SABS_1516_37155_Middle.shp"
middle_sabs_gdf = gpd.read_file(filename)
middle_sabs_gdf.head()

,SrcName,ncessch,schnam,leaid,gslo,gshi,defacto,stAbbrev,openEnroll,Shape_Leng,Shape_Area,level,MultiBdy,slcncessch,slcleaid,geometry
0,07-05-13-SGM-13-14,370225003249,Sandy Grove Middle,3702250,06,08,0,NC,0,78318.730737,2.776846e+08,2,0,1,0,"POLYGON ((-79.11123 35.01107, -79.11112 35.011..."
1,Fairgrove Middle,370393001570,Fairgrove Middle,3703930,04,08,0,NC,0,109097.375184,2.644877e+08,2,0,1,1,"POLYGON ((-79.12690 34.61874, -79.12567 34.618..."
2,Littlefield Middle,370393001572,Littlefield Middle,3703930,04,08,0,NC,0,122694.419075,4.393298e+08,2,0,1,1,"POLYGON ((-78.84945 34.73266, -78.84932 34.732..."
3,Orrum Middle,370393001575,Orrum Middle,3703930,05,08,0,NC,0,138415.487805,3.849775e+08,2,0,1,1,"POLYGON ((-79.02042 34.59294, -79.00594 34.586..."
4,Pembroke Middle,370393001579,Pembroke Middle,3703930,06,08,0,NC,0,114981.291127,4.064122e+08,2,0,1,1,"POLYGON ((-79.09406 34.76784, -79.09148 34.756..."


In [11]:
middle_sabs_gdf['ncessch'].describe()

count               12
unique              12
top       370393002240
freq                 1
Name: ncessch, dtype: object

In [12]:
filename = sourcefolder+"/"+sourceprogram+"/"+"SABS_1516_37155_Primary.shp"
primary_sabs_gdf = gpd.read_file(filename)
primary_sabs_gdf.head()

,SrcName,ncessch,schnam,leaid,gslo,gshi,defacto,stAbbrev,openEnroll,Shape_Leng,Shape_Area,level,MultiBdy,slcncessch,slcleaid,geometry
0,Deep Branch Elem,370393001569,Deep Branch Elementary,3703930,PK,06,0,NC,0,52550.774635,8.788543e+07,1,0,1,1,"POLYGON ((-79.09371 34.68968, -79.09047 34.685..."
1,Green Grove Elem,370393001571,Green Grove Elementary,3703930,PK,03,0,NC,0,109097.375184,2.644877e+08,1,0,1,1,"POLYGON ((-79.12690 34.61874, -79.12567 34.618..."
2,Long Branch Elem,370393001573,Long Branch Elementary,3703930,PK,04,0,NC,0,138415.487805,3.849775e+08,1,0,1,1,"POLYGON ((-79.02042 34.59294, -79.00594 34.586..."
3,Magnolia Elem KI - 06,370393001574,Magnolia Elementary,3703930,PK,08,0,NC,0,102401.733197,2.627364e+08,1,0,1,1,"POLYGON ((-79.01120 34.73224, -79.00600 34.728..."
4,Oxendine Elem,370393001576,Oxendine Elementary,3703930,PK,06,0,NC,0,63919.056252,1.160995e+08,1,0,1,1,"POLYGON ((-79.34600 34.83850, -79.34587 34.838..."


In [13]:
primary_sabs_gdf['ncessch'].describe()

count               23
unique              23
top       370393002247
freq                 1
Name: ncessch, dtype: object

## Spatial Join Polygons to Points

Original code idea from https://geoffboeing.com/2016/10/r-tree-spatial-index-python/

In [14]:
def spatial_join_polygons_to_points(points_gdf, polygon_gdf,vardict_list):
    # Ensure both points and polygons have the same CRS
    from pyproj import CRS
    points_gdf.crs = CRS("epsg:4326")
    polygon_gdf.crs = CRS("epsg:4326")

    # build the r-tree index - for Points
    sindex_points_gdf = points_gdf.sindex

    # find the points that intersect with each subpolygon
    for index, polygon in polygon_gdf.iterrows():
        # find approximate matches with r-tree, then precise matches from those approximate ones
        possible_matches_index = list(sindex_points_gdf.intersection(polygon['geometry'].bounds))
        possible_matches = points_gdf.iloc[possible_matches_index]
        precise_matches = possible_matches[possible_matches.intersects(polygon['geometry'])]
        
        # join variables from polygon to point
        for sourcevar in vardict_list:
            name_of_join_var = vardict_list[sourcevar]
            points_gdf.loc[precise_matches.index,name_of_join_var] = polygon[sourcevar]
            
    return points_gdf

In [15]:
block_high_gdf = spatial_join_polygons_to_points(block_gdf, 
                                                 high_sabs_gdf,
                                                 {'ncessch':'ncessch_3','schnam':'high_schnm'})
block_hmid_gdf = spatial_join_polygons_to_points(block_high_gdf,
                                                 middle_sabs_gdf,
                                                 {'ncessch':'ncessch_2','schnam':'mid_schnm'})
block_asabs_gdf = spatial_join_polygons_to_points(block_hmid_gdf,
                                                  primary_sabs_gdf,
                                                  {'ncessch':'ncessch_1','schnam':'primary_schnm'})

block_asabs_gdf.head()

,Unnamed: 0,STATEFP10,COUNTYFP10,TRACTCE10,BLOCKCE,BLOCKID10,PARTFLG,HOUSING10,POP10,geometry,...,PLCGEOID10,PLCNAME10,PUMGEOID10,PUMNAME10,ncessch_3,high_schnm,ncessch_2,mid_schnm,ncessch_1,primary_schnm
0,0,37,155,961900,2028,371559619002028,N,14,52,POINT (-79.22459 34.45879),...,NaN,NaN,3705100,Robeson County (West)--Lumberton City PUMA,370393002184,South Robeson High,370393001570,Fairgrove Middle,370393001571,Green Grove Elementary
1,1,37,155,961900,2054,371559619002054,N,1,3,POINT (-79.18141 34.40608),...,NaN,NaN,3705100,Robeson County (West)--Lumberton City PUMA,370393002232,Fairmont High,370393002233,Fairmont Middle,370393002241,Rosenwald Elementary
2,2,37,155,961700,2069,371559617002069,N,41,99,POINT (-79.16201 34.48768),...,NaN,NaN,3705100,Robeson County (West)--Lumberton City PUMA,370393002232,Fairmont High,370393002233,Fairmont Middle,370393002241,Rosenwald Elementary
3,3,37,155,961700,2065,371559617002065,N,6,22,POINT (-79.16259 34.50130),...,NaN,NaN,3705100,Robeson County (West)--Lumberton City PUMA,370393002232,Fairmont High,370393002233,Fairmont Middle,370393002241,Rosenwald Elementary
4,4,37,155,961700,2058,371559617002058,N,19,55,POINT (-79.14702 34.49788),...,NaN,NaN,3705100,Robeson County (West)--Lumberton City PUMA,370393002232,Fairmont High,370393002233,Fairmont Middle,370393002241,Rosenwald Elementary


In [16]:
block_asabs_gdf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [17]:
block_asabs_gdf['BLOCKID10'].astype(str).describe()

count                5799
unique               5799
top       371559620022008
freq                    1
Name: BLOCKID10, dtype: object

In [18]:
block_asabs_gdf.loc[block_asabs_gdf['high_schnm'].isna(), 'high_schnm'] = 'No High School SAB'
block_asabs_gdf.loc[block_asabs_gdf['mid_schnm'].isna(), 'mid_schnm'] = 'No Middle School SAB'
block_asabs_gdf.loc[block_asabs_gdf['primary_schnm'].isna(), 'primary_schnm'] = 'No Primary School SAB'

In [19]:
pd.pivot_table(block_asabs_gdf, 
               values = ['BLOCKID10','POP10'],
               index=['high_schnm','mid_schnm','primary_schnm'], 
               aggfunc={'BLOCKID10':'count','POP10':'sum'}, 
               margins=True, margins_name = 'Total')

BLOCKID10  \
high_schnm            mid_schnm             primary_schnm                           
Fairmont High         Fairmont Middle       Rosenwald Elementary              478   
                      Orrum Middle          Long Branch Elementary            320   
Lumberton Senior High Fairgrove Middle      Green Grove Elementary             27   
                      Littlefield Middle    East Robeson Primary              303   
                      Lumberton Junior High Janie C Hargrave Elem             213   
                                            Rowland Norment Elementary        274   
                                            Tanglewood Elementary             419   
                                            W H Knuckles                      104   
                                            West Lumberton Elementary         147   
                      No Middle School SAB  Magnolia Elementary               139   
                                            Piney Grove Elementary            130   
                      Orrum Middle          Long Branch Elementary             16   
Purnell Swett High    No Middle School SAB  Piney Grove Elementary             12   
                                            Prospect Elementary               153   
                      Pembroke Middle       Deep Branch Elementary             90   
                                            Pembroke Elementary               301   
                                            Union Chapel Elementary            69   
                                            Union Elementary                  104   
                      Townsend Middle       R B Dean Elementary               258   
Red Springs High      No Middle School SAB  Prospect Elementary                10   
                      Pembroke Middle       Union Chapel Elementary            85   
                      Red Springs Middle    Oxendine Elementary               194   
                                            Peterson Elementary               282   
                                            Rex-Rennert Elementary             98   
                      Townsend Middle       R B Dean Elementary                10   
Saint Pauls High      No Middle School SAB  Magnolia Elementary                83   
                                            Parkton Elementary                258   
                                            Piney Grove Elementary             16   
                                            Rex-Rennert Elementary              1   
                      Saint Pauls Middle    Rex-Rennert Elementary             90   
                                            Saint Pauls Elementary            393   
South Robeson High    Fairgrove Middle      Green Grove Elementary            251   
                      Pembroke Middle       Deep Branch Elementary             12   
                                            Union Elementary                   43   
                      Rowland Middle        Southside/Ashpole Elem            343   
                      Townsend Middle       R B Dean Elementary                73   
Total                                                                        5799   

                                                                         POP10  
high_schnm            mid_schnm             primary_schnm                       
Fairmont High         Fairmont Middle       Rosenwald Elementary          9473  
                      Orrum Middle          Long Branch Elementary        6707  
Lumberton Senior High Fairgrove Middle      Green Grove Elementary         623  
                      Littlefield Middle    East Robeson Primary          9423  
                      Lumberton Junior High Janie C Hargrave Elem         3303  
                                            Rowland Norment Elementary    6620  
                                            Tanglewood Elementary         8032  
                           

In [21]:
# Save Work at this point as CSV
columns =['BLOCKID10','ncessch_3','high_schnm','ncessch_2','mid_schnm','ncessch_1','primary_schnm']
savefile = sys.path[0]+"/"+programname+"/"+programname+".csv"
block_asabs_gdf.to_csv(savefile, columns = columns, index = False)